# Convertir etiquetado personalizado a formato YOLO Darknet

> Input
*   Directorio que contiene las imagenes y el archivo de etiquetado en formato JSON
*   Ancho de las imagenes del dataset 
*   Alto de las imagenes del dataset 

> Output
* Carpeta 'labels' con etiquetas del dataset en formato YOLO Darknet (un archivo TXT con etiquetas para cada imagen)


In [27]:
#parametros iniciales necesarios

directorio = '/content/drive/MyDrive/PT/dataset/IMG_1123'    #directorio que contiene las imagenes y el archivo de etiquetado en formato json
w_i = 1920    #ancho de imagenes en entero, ej: 3264
h_i = 1080    #alto de imagenes en entero, ej: 2448

carpeta = 'labels'    #carpeta donde se almacenara el nuevo etiquetado generado     

In [28]:
#importacion de bibliotecas necesarias

import json
import os
import errno
from PIL import Image
from PIL.ExifTags import TAGS   

In [29]:
#definicion de variables 

w = w_i
h = h_i    #se asignan valores recibidos de ancho y alto a variables auxiliares 
directorio = directorio.replace('\\', '/') + "/"    #se modifica el directorio para correcta lectura de python
contenido_directorio = os.listdir(directorio)
data = {}
nombre_archivos_jpg = []        
ejpg = '.jpg'
ejson = '.json'
etxt = '.txt'
#archivo_json = 'berries.json'    #nombre del archivo que contiene el etiquetado original

In [30]:
#comprobacion de archivos en directorio

try:
    os.mkdir(directorio + carpeta)    #se crea una carpeta para almacenar todos los archivos de etiquetado creados
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

for archivo in contenido_directorio:
    if os.path.isfile(os.path.join(directorio, archivo)) and archivo.endswith(ejpg):
        nombre_archivos_jpg.append(archivo.replace(ejpg, ''))    #se obtienen los nombres de los archivos JPG encontrados y se almacenan en una lista sin la extension '.jpg'
        print("Archivo JPG encontrado: ", archivo)
        
for archivo in contenido_directorio:
    if os.path.isfile(os.path.join(directorio, archivo)) and archivo.endswith(ejson):
      archivo_json = archivo    #se obtiene el nombre del primer archivo JSON encontrado y se almacena en la variable archivo_json
      print("Archivo JSON encontrado: ", archivo)
      break

etiquetas = directorio + archivo_json

Archivo JPG encontrado:  frame0000.jpg
Archivo JPG encontrado:  frame0111.jpg
Archivo JPG encontrado:  frame0127.jpg
Archivo JPG encontrado:  frame0135.jpg
Archivo JPG encontrado:  frame0191.jpg
Archivo JPG encontrado:  frame0206.jpg
Archivo JPG encontrado:  frame0219.jpg
Archivo JPG encontrado:  frame0232.jpg
Archivo JPG encontrado:  frame0239.jpg
Archivo JPG encontrado:  frame0269.jpg
Archivo JPG encontrado:  frame0281.jpg
Archivo JPG encontrado:  frame0298.jpg
Archivo JPG encontrado:  frame0358.jpg
Archivo JPG encontrado:  frame0371.jpg
Archivo JPG encontrado:  frame0386.jpg
Archivo JPG encontrado:  frame0393.jpg
Archivo JPG encontrado:  frame0404.jpg
Archivo JPG encontrado:  frame0440.jpg
Archivo JPG encontrado:  frame0458.jpg
Archivo JPG encontrado:  frame0469.jpg
Archivo JPG encontrado:  frame0479.jpg
Archivo JPG encontrado:  frame0516.jpg
Archivo JPG encontrado:  frame0544.jpg
Archivo JPG encontrado:  frame0582.jpg
Archivo JPG encontrado:  frame0607.jpg
Archivo JPG encontrado:  

In [31]:
#generacion de etiquetas

with open(etiquetas) as file:
    data = json.load(file)

    for nombre in nombre_archivos_jpg:    #para cada nombre almacenado en 'nombre_archivos_jpg', obtenidos a partir del dataset de imagenes
        puntos = []
        lineas = []
        archivo_txt = directorio + carpeta + '/' + nombre + etxt    #se genera un archivo '.txt' para almacenar el etiquetado en formato 'YOLO DARKNET TXT'
        imagen = nombre + ejpg    #imagen es el "objeto" a buscar en el json (nombre de la imagen, cuyo etiquetado esta contenido en el objeto)
        archivo_jpg = directorio + imagen    #ubicacion de cada imagen para lectura de metadatos
        image = Image.open(archivo_jpg)    #Lee la imagen en el directorio señalado
        exifdata = image.getexif() 

        for tagid in exifdata: 
            tagname = TAGS.get(tagid, tagid) 
            value = exifdata.get(tagid) 
            if (tagname == "Orientation" and value == 1):    #si la orientacion de la imagen es horizontal
                w = w_i
                h = h_i
            if (tagname == "Orientation" and value == 6):    #si la orientacion de la imagen es vertical
                w = h_i
                h = w_i
            #print(f"{tagname:25}: {value}") 
        image.close()

        try:    #se usa try-except en caso de que no exista etiquetado en el json para alguna imagen del dataset
            for objeto in data[imagen]:
                puntos.append("0")    #se agrega a la lista un 0 (clase a la que pertenece el etiquetado), y luego todos los puntos del bounding box

                punto_x = int(objeto['x'])    #los puntos obtenidos del archivo json se transforman a formato numerico
                punto_y = int(objeto['y'])
                punto_w = int(objeto['w'])
                punto_h = int(objeto['h'])

                bbox_w = (punto_w - punto_x)/w    #coordenadas normalizadas segun requerimientos de YOLO 
                bbox_h = (punto_h - punto_y)/h
                bbox_x = (punto_x/w + bbox_w/2)
                bbox_y = (punto_y/h + bbox_h/2)

                puntos.append(str("{0:.6f}".format(bbox_x)))    #se limita la cantidad de decimales en las coordenadas normalizadas 
                puntos.append(str("{0:.6f}".format(bbox_y)))    #y se almacenan en una lista como string
                puntos.append(str("{0:.6f}".format(bbox_w))) 
                puntos.append(str("{0:.6f}".format(bbox_h))) 
                
                lineas.append(" ".join(puntos))    #la lista de puntos previa es un bounding box y este se almacenan en otra lista
                puntos = []

        except KeyError:    #se controla excepcion en caso de que no exista etiquetado en json para alguna imagen del dataset
            lineas = ""    #se eliminan los puntos almacenados del archivo anterior
            archivo_txt = ""    #se elimina el nombre del archivo sin etiquetado
            print("Error en: ", imagen)

        if archivo_txt != "":    #no se consideran los nombres de archivo sin etiquetado
            print("======================================")
            print("Imagen: ", archivo_jpg)
            print("Lineas generadas (etiquetas): ")
            with open(archivo_txt, "w") as file:
                for linea in lineas:
                    print(linea)
                    file.write(linea + "\n")    #se escriben los puntos de los bounding box en el archivo '.txt'
            print("Archivo de texto generado: ", archivo_txt)
            print("======================================")

            WINDOWS_LINE_ENDING = b'\r\n'
            UNIX_LINE_ENDING = b'\n'
            with open(archivo_txt, 'rb') as open_file:
                content = open_file.read()
                content = content.replace(WINDOWS_LINE_ENDING, UNIX_LINE_ENDING)    #conversion de formato de endline windows->unix
            with open(archivo_txt, 'wb') as open_file:
                open_file.write(content)

Se han truncado las últimas 5000 líneas del flujo de salida.
0 0.396875 0.293519 0.044792 0.075926
0 0.379167 0.032407 0.078125 0.064815
0 0.343229 0.450926 0.060417 0.105556
0 0.318229 0.356481 0.047917 0.087037
0 0.288542 0.292593 0.055208 0.051852
0 0.289062 0.226852 0.050000 0.079630
0 0.243750 0.098148 0.076042 0.133333
0 0.179688 0.137037 0.083333 0.133333
0 0.178646 0.226852 0.079167 0.101852
0 0.143750 0.097222 0.076042 0.135185
0 0.037500 0.228704 0.073958 0.120370
0 0.100521 0.036111 0.041667 0.072222
0 0.797917 0.200000 0.056250 0.092593
0 0.836979 0.221296 0.028125 0.050000
0 0.872917 0.245370 0.050000 0.075926
0 0.874479 0.321296 0.055208 0.090741
0 0.847396 0.269444 0.036458 0.064815
0 0.816667 0.259259 0.039583 0.066667
0 0.779167 0.279630 0.047917 0.085185
0 0.745312 0.267593 0.032292 0.053704
0 0.788542 0.328704 0.039583 0.061111
0 0.799479 0.384259 0.034375 0.061111
0 0.750521 0.325926 0.034375 0.059259
0 0.765104 0.374074 0.048958 0.081481
0 0.631250 0.347222 0.02708